In [ ]:
import pandas as pd
from sklearn.metrics import f1_score, recall_score, precision_score, confusion_matrix, roc_auc_score
import numpy as np

In [ ]:
df = pd.read_csv('FFfull.csv', low_memory=False)

web_df = df[df['study'] == 'web'].copy()
field_df = df[df['study'] == 'field'].copy()

song_functions = df.groupby("song").agg({'songfunction': 'first'})
web_ratings = web_df[["songfunction", "song", "danc","heal","baby","love"]]

In [ ]:
# Evil bit hacking
arr = web_ratings[["danc","heal","baby","love"]].to_numpy()
max_values = np.max(arr, axis=1, keepdims=True)
mask = (arr == max_values)
random_masked = np.random.random(arr.shape) * mask
web_ratings["y_pred"] = np.argmax(random_masked, axis=1)

In [ ]:
# Create ordered categorical with your specific order
ordered_cats = pd.CategoricalDtype(categories=['danc', 'heal', 'baby', 'love'], ordered=True)
series_cat = web_ratings.songfunction.astype(ordered_cats)

# Now the codes will match your order
web_ratings["y_true"] = series_cat.cat.codes

### Humans vs. Ground Truth

In [ ]:
y_pred = web_ratings.y_pred.to_numpy()
y_true = web_ratings.y_true.to_numpy()
cm = confusion_matrix(y_true, y_pred)

def compute_metrics(y_test, y_pred):
    """Compute classification metrics."""
    metrics = {
        "f1": f1_score(y_test, y_pred, average='macro'),
        "recall": recall_score(y_test, y_pred, average='macro'),
        "precision": precision_score(y_test, y_pred, average='macro'),
        "per_class": {
            "precision": precision_score(y_test, y_pred, average=None).tolist(),
            "recall": recall_score(y_test, y_pred, average=None).tolist(),
            "f1": f1_score(y_test, y_pred, average=None).tolist()
        }
    }
        
    return metrics


metrics = compute_metrics(y_true, y_pred)

In [ ]:
from pprint import pprint
pprint(metrics)
print("Confusion Matrix")
pprint(cm)